# CHATBOT

## Intro
* Chatbots are really helpful to make our lives easy. This chatbot saves two types of memory, a basic on-going conversation and then the summary of the entire conversation as permanent memory in the storage file.

## The Problems
* Langchain documentations are very confusing, We have to make the chatbot give contexual response and then remember it. Also not becoming so memory expensive by saving entire responses

## The process we will follow
1. Create two memories, Temproray and permanent.
2. Create a function that stores user and assistant responses in the temprorary and permanent memory.
3. Then retrieves it on the basis of the specific user ID.
4. Respond to the current question with context to temprorary as well as permanent summary.
5. Update the vectorstore or the database with the new conversation as well.

## Setup

#### After you download the code from the github repository in your computer
In terminal:
* cd project_name
* pyenv local 3.11.4
* poetry install
* poetry shell

#### To open the notebook with Jupyter Notebooks
In terminal:
* jupyter lab

Go to the folder of notebooks and open the right notebook.

#### To see the code in Virtual Studio Code or your editor of choice.
* open Virtual Studio Code or your editor of choice.
* open the project-folder
* open the Men2.py file

## Create your .env file
* In the github repo we have included a file named .env.example
* Rename that file to .env file and here is where you will add your confidential api keys. Remember to include:
* OPENAI_API_KEY=your_openai_api_key
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
* LANGCHAIN_API_KEY=your_langchain_api_key
* LANGCHAIN_PROJECT=your_project_name

## Connect with the .env file located in the same directory of this notebook

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [1]:
#!pip install python-dotenv


In [8]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

## Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [9]:
#!pip install langchain


## Connect with an LLM

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [10]:
#!pip install langchain-openai

In [11]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

We will use converation buffer memory to initialize conversation memory to save on-going conversation to answer contextually. And a summary memory which will be also be saving the responses in a summarized manner.

# FAISS

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory

summary_memory = ConversationBufferMemory(llm = llm, max_token_limit = 500)
convo_memory = ConversationBufferMemory(llm = llm, max_token_limit = 500)

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
def load_faiss_index(file_path="faiss_index"):
    return FAISS.load_local(file_path, embeddings, allow_dangerous_deserialization=True)
# Add an instruction to guide the assistant
system_instruction = "You are a nice, helpful assistant"

vectorstore = FAISS.from_texts([""], embedding=embeddings)

# FAISS Index Inspection and User Summary Retrieval
This Python code consists of two functions designed to interact with a FAISS-based vector store. The FAISS index is used for efficient similarity searches in vectorized data, such as text embeddings.

1. **inspect_faiss_index(vectorstore)**
Purpose:
This function inspects the content stored in a FAISS index, including its associated metadata.

How It Works:
It accesses the docstore dictionary of the vectorstore to retrieve stored texts and their metadata.
It iterates through each item in the dictionary and prints:
ID: The unique identifier for the stored text.
Text: The actual page content stored in the FAISS index.
Metadata: Additional information associated with the stored text, such as user-specific data or type.

2. **retrieve_user_summary(vectorstore, user_id)**
Purpose:
This function retrieves the latest summary for a specific user based on their user_id.

How It Works:
It uses the similarity_search() method of the vectorstore to search for a record that matches:
user_id (the specific user identifier).
type set to "summary" (indicating that the data is a user summary).
It retrieves only one result (k=1) and returns the page_content (text) of the result if found.
If no matching record is found, it returns an empty string ("").

In [ ]:

def inspect_faiss_index(vectorstore):
    """
    Inspect the texts and metadata stored in the FAISS index.
    """
    # Retrieve stored texts and their metadata
    stored_texts = vectorstore.docstore._dict  # Access stored texts
    for key, value in stored_texts.items():
        print(f"ID: {key}")
        print(f"Text: {value.page_content}")
        print(f"Metadata: {value.metadata}")
        print("-" * 50)
def retrieve_user_summary(vectorstore, user_id):
    """Retrieve the latest summary for a specific user."""
    results = vectorstore.similarity_search("", k=1, filter={"user_id": user_id, "type": "summary"})
    return results[0].page_content if results else ""


# FAISS Integration and Conversation Management Code Explanation:

This code provides functionality to manage and store conversations using FAISS (Facebook AI Similarity Search). It also includes methods for generating summaries, interacting with an LLM (Language Model), and managing the FAISS index itself.

## 1. Storing Final Summaries

This function generates and stores a final summary of a conversation into the FAISS index. Here's the process:

Conversation Retrieval: The conversation history is retrieved using summary_memory.load_memory_variables({}), where the history key contains the entire conversation.

Summary Creation: An LLM is instructed to summarize the conversation using a defined summary_instruction and a summary_prompt containing the conversation history.

FAISS Storage: The generated summary (final_summary) is stored in the FAISS index, along with metadata specifying the user_id and data type ("summary").

Output: The function prints confirmation that the summary has been stored successfully.

In [15]:
summary_instruction = "You are a concise assistant. Generate a brief summary of the conversation, and who gives contexual answers using the summary stored and the previous recent messages"
def store_final_summary(vectorstore, user_id):
    """Summarize the entire conversation in memory and store it."""
    # Retrieve the entire conversation from memory
    conversation_history = summary_memory.load_memory_variables({})["history"]

    # Use LLM to summarize the conversation
    summary_prompt = f"{summary_instruction}\n\nConversation History:\n{conversation_history}\n\nSummary:"
    final_summary = llm.invoke(summary_prompt).content.strip()

    # Store the summary in FAISS
    vectorstore.add_texts(
        [final_summary],
        metadatas=[{"user_id": user_id, "type": "summary"}]
    )
    print("\nConversation summarized and stored in FAISS.")

## 2. Summarization and Response Handling

This function manages user interactions while summarizing and maintaining a conversation flow:

Summary Retrieval: It retrieves an existing summary for the user by invoking the retrieve_user_summary function.

User Input Management: The current user input is added to the chat_memory.

Full Prompt Construction: A prompt is constructed by combining:
 1.A predefined system_instruction.
 2.The retrieved existing_summary.
 3.The current user_input.

Response Generation: The LLM generates a response using the constructed prompt, which is then added back to the conversation memory (chat_memory).

Output: The function returns the AI's response to be sent back to the user

In [16]:
def summarize_and_answer(vectorstore, user_id, user_input):
    existing_summary = retrieve_user_summary(vectorstore, user_id)
    convo_memory.chat_memory.add_user_message(user_input)
    full_prompt = f"{system_instruction}\n{existing_summary}\nUser: {user_input}\n{convo_memory}\nAI:"
    response = llm.invoke(full_prompt).content
    summary_memory.chat_memory.add_user_message(user_input)
    summary_memory.chat_memory.add_ai_message(response)

    return response

## 3. Saving the FAISS Index Locally

This function allows saving the current FAISS index to a local file:

Saving Process: The save_local() method is invoked with the specified file_path.

Confirmation: A message confirms that the FAISS index has been saved.

This is particularly useful for preserving the FAISS database between sessions.

In [17]:
def save_faiss_Index(vectorstore, file_path = "faiss_index"):
    vectorstore.save_local(file_path)
    print("Chat saved")
    print(f"FAISS index saved at: {file_path}")
    return

## 4. Clearing the FAISS Index

This function resets the FAISS index to an empty state:

Index Reset: A new empty FAISS index is created using FAISS.from_texts([""], embedding=embeddings).

Optional Save: The index can be saved after clearing (commented out in the code).

Output: A message confirms that all data has been deleted.

This is useful for cleaning up or restarting the FAISS database

In [18]:
def clear_faiss_index(file_path="faiss_index"):
    """Clear the FAISS index by replacing it with an empty one."""
    global vectorstore
    vectorstore = FAISS.from_texts([""], embedding=embeddings)
    # save_faiss_Index(vectorstore, file_path)
    print("All data has been deleted from the FAISS database.")

# Managing FAISS Index and Interactive User Input Loop:

This code snippet demonstrates how to manage a FAISS index, handle user inputs in an interactive loop, and provide responses using a conversational AI system.

## 1. Loading the FAISS Index
1. Purpose: 
Attempt to load a previously saved FAISS index from disk.

2. Error Handling: 
If the FAISS index does not exist or cannot be loaded, an exception is caught, and a message indicates that a new FAISS index will be initialized.

Outcome: Ensures the system always has a vectorstore instance, either loaded or newly created.

In [19]:
try:
    vectorstore = load_faiss_index(file_path="faiss_index")
    print("Loaded FAISS index from disk.")
except Exception as e:
    print(f"No existing FAISS index found. Initializing a new one.")

No existing FAISS index found. Initializing a new one.


## 2. User Input and Interaction:

1. User Identification: The user_id input identifies the specific user for whom the data is being processed and stored.

2. Input Loop: A while loop continuously prompts the user to provide input until a specific condition (e.g., "exit" or "delete data") is met.

In [ ]:
user_id = input("Enter user_id :")
while True:
    user_input = input("Say something: ")
    if user_input == "exit":
        store_final_summary(vectorstore, user_id)
        break
    if user_input == "delete data":
        clear_faiss_index()
        break
    response = summarize_and_answer(vectorstore, user_id, user_input)
    print(f"AI: {response}")
